In [1]:
print('test')

test


In [ ]:
# corerenza di utilizzo nomi per etichette

#### label_id    ( etichetta codice, es:  "A03"  )
###  label_text  ( etichetta testuale, es:  "Colera"  )
###  source_text ( testo finale da embeddare: "Colera, malattia colerica ..."" )

## Labels

### Labels data cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt


malat_Cat = pd.read_excel("../data_icd10/Codici-ICD10-categorie_mlt_tmt.xlsx", sheet_name='categorie_mlt_tmt',header=1)
malat_SubCat = pd.read_excel("../data_icd10/Codici-ICD10-categorie_mlt_tmt.xlsx", sheet_name='sottocategorie_mlt_tmt',header=1)

##############################
# Rename columns, but keep old names:
column_name_reference_CAT = {
    "Codice Categoria Malattie e Traumatismi": "Codice_categoria",
    "Capitolo": "Capitolo",
    "Blocchi di categorie": "Blocco",
    "Descrizione delle categorie di Malattie e traumatismi": "Descr_Categ"
}
# Apply renaming
malat_Cat = malat_Cat.rename(columns=column_name_reference_CAT)

column_name_reference_SubCat = {
    "Codice Categoria Malattie e traumatismi": "Codice_categoria",
    "Codice sottocategoria Malattie e Traumatismi": "Codice_SottoCategoria",
    "Descrizione delle sottocategorie di Malattie e traumatismi": "Descr_SottoCateg"
}

#
malat_SubCat = malat_SubCat.rename(columns=column_name_reference_SubCat)

malat_SubCat['Codice_SottoCategoria']= malat_SubCat['Codice_SottoCategoria'].astype('Int64')
malat_Cat['Capitolo'] = malat_Cat['Capitolo'].astype('Int64')
malat_Cat['Blocco'] = malat_Cat['Blocco'].astype('Int64')

# elimina righe vuote
malat_Cat = malat_Cat.dropna()
malat_SubCat = malat_SubCat.dropna()

display(malat_Cat.head(3))
print('-------------')
display(malat_SubCat.head(7))




,Codice_categoria,Capitolo,Blocco,Descr_Categ
0,A00,1,1,Colera
1,A01,1,1,Febbre tifoide e paratifo
2,A02,1,1,Altre infezioni da salmonella


-------------


,Codice_categoria,Codice_SottoCategoria,Descr_SottoCateg
0,A00,0,"Colera da Vibrio cholerae 01, biotipo del colera"
1,A00,1,"Colera da Vibrio cholerae 01, biotipo eltor"
2,A00,9,Colera non specificato
3,A01,0,Febbre tifoide
4,A01,1,Paratifo A
5,A01,2,Paratifo B
6,A01,3,Paratifo C


In [2]:
# Goal is to concatenate all subcat text for each categ.
# Concatenate text in each category with groupby and join

df_subcat_text= malat_SubCat.groupby('Codice_categoria')['Descr_SottoCateg'].apply(lambda text: ' '.join(text))

# and also remake it a proper dataframe with reset index, 
# so that it can be merged back
df_subcat_text= df_subcat_text.reset_index()
df_subcat_text.columns = ['Codice_categoria', 'Descr_SottoCateg_Estesa']

display(malat_SubCat.head(5))
df_subcat_text.head(3)

,Codice_categoria,Codice_SottoCategoria,Descr_SottoCateg
0,A00,0,"Colera da Vibrio cholerae 01, biotipo del colera"
1,A00,1,"Colera da Vibrio cholerae 01, biotipo eltor"
2,A00,9,Colera non specificato
3,A01,0,Febbre tifoide
4,A01,1,Paratifo A


,Codice_categoria,Descr_SottoCateg_Estesa
0,A00,"Colera da Vibrio cholerae 01, biotipo del cole..."
1,A01,Febbre tifoide Paratifo A Paratifo B Paratifo ...
2,A02,Enterite da salmonella Setticemia da salmonell...


In [3]:
#### merge

malat_Cat_full = malat_Cat.merge(
    how='left',
    right= df_subcat_text,
    on= 'Codice_categoria',
)

# sostituisce NaN con stringa vuota per fare unione finale stringhe
malat_Cat_full['Descr_SottoCateg_Estesa'] =  malat_Cat_full['Descr_SottoCateg_Estesa'].fillna('')
malat_Cat_full.tail()

# crea descrizione totale unendo anche descrizione base della categoria 
malat_Cat_full['Descr_totale_Estesa'] =  malat_Cat_full['Descr_Categ'] + ': ' + malat_Cat_full['Descr_SottoCateg_Estesa']
malat_Cat_full.tail(3)

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
1949,Y96,20,35,Condizione associata al lavoro,Condizione associata al lavoro,Condizione associata al lavoro: Condizione ass...
1950,Y97,20,35,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambienta...
1951,Y98,20,35,Condizione associata allo stile di vita,Condizione associata allo stile di vita,Condizione associata allo stile di vita: Condi...


#### Final Cleaned Dataframe to use for collection

In [10]:
malat_Cat_full.head()

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
0,A00,1,1,Colera,"Colera da Vibrio cholerae 01, biotipo del cole...","Colera: Colera da Vibrio cholerae 01, biotipo ..."
1,A01,1,1,Febbre tifoide e paratifo,Febbre tifoide Paratifo A Paratifo B Paratifo ...,Febbre tifoide e paratifo: Febbre tifoide Para...
2,A02,1,1,Altre infezioni da salmonella,Enterite da salmonella Setticemia da salmonell...,Altre infezioni da salmonella: Enterite da sal...
3,A03,1,1,Shigellosi,Shigellosi da Shigella dysenteriae Shigellosi ...,Shigellosi: Shigellosi da Shigella dysenteriae...
4,A04,1,1,Altre infezioni batteriche intestinali,Infezione da Escherichia coli enteropatogena I...,Altre infezioni batteriche intestinali: Infezi...


### Labels create collection

In [4]:
# Dai puliti:
malat_Cat_full.tail(3)

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
1949,Y96,20,35,Condizione associata al lavoro,Condizione associata al lavoro,Condizione associata al lavoro: Condizione ass...
1950,Y97,20,35,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambienta...
1951,Y98,20,35,Condizione associata allo stile di vita,Condizione associata allo stile di vita,Condizione associata allo stile di vita: Condi...


In [ ]:
# per usare i moduli 
# da temrinal fare pip install -e . 
# ( "e"  sta per editing )
# cosi installa tutto e perfmettte di fare modifiche ai moduli e riavviare e riusarli



In [5]:
from semantic_search.data import Document 

docs = [
    Document(id=1, content="Apple",  metadata={"type":"fruit"}),
    Document(id=2, content="Banana", metadata={"type":"fruit"}),
    Document(id=3, content="Car",    metadata={"type":"vehicle"}),
    Document(id=4, content="Bicycle",metadata={"type":"vehicle"}),
    Document(id=5, content="Dog",    metadata={"type":"animal"}),
    Document(id=6, content="Cat",    metadata={"type":"animal"}),
    Document(id=7, content="Paris",  metadata={"type":"city"}),
    Document(id=8, content="New York",metadata={"type":"city"}),
    Document(id=9, content="Python", metadata={"type":"programming_language"}),
    Document(id=10, content="Java",  metadata={"type":"programming_language"}),
]

docs[0]

Document(id=1, content='Apple', metadata={'type': 'fruit'})

## Test-Data

### Test-Data Cleaning

### Test-Data Results